In [33]:
import requests
from fastapi import FastAPI,Header
from pydantic import BaseModel
from nest_asyncio import apply
import uvicorn

from typing import Any
from fastapi.responses import RedirectResponse
from fastapi.responses import ORJSONResponse

In [13]:
from fastapi.encoders import jsonable_encoder
from fastapi.responses import HTMLResponse,PlainTextResponse
from fastapi.responses import Response,PlainTextResponse
from fastapi.responses import StreamingResponse

In [ ]:
app = FastAPI()

CHUNK_SIZE = 1024*1024 # 1 Mb per chunk
path = '../temp/sample.mp4'

async def read_video_chunks(file_path):
    with open(file_path, "rb") as video_file:
        while True:
            chunk = video_file.read(CHUNK_SIZE)
            if not chunk:
                break
            yield chunk
        
@app.get("/")
async def main():
    return StreamingResponse(read_video_chunks(path) ,media_type="video/mp4")

if __name__ == "__main__":
    apply()
    uvicorn.run(app,port=4444)

INFO:     Started server process [788506]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:52840 - "GET /index/ HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:52846 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52848 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:59754 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:59768 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:34198 - "GET / HTTP/1.1" 200 OK


In [39]:
video_path = '../temp/sample.mp4'

CHUNK_SIZE = 1048 * 1048   # 1 MB chunk size

@app.get("/video")
async def video_endpoint(range: str = Header(None)):
    start, end = range.replace("bytes=", "").split("-")
    start = int(start)
    end = int(end) if end else start + CHUNK_SIZE
    filesize = str(video_path.stat().st_size)
    print(filesize)
    headers = {
        'Content-Range': f'bytes {start}-{end-1}/{filesize}',
        'Accept-Ranges': 'bytes'
    }

    def video_stream():
        with open(video_path, "rb") as video:
            video.seek(start)
            while True:
                chunk = video.read(CHUNK_SIZE)
                if not chunk:
                    break
                yield chunk

    return StreamingResponse(video_stream(), media_type="video/mp4", status_code=206, headers=headers)

if __name__ == "__main__":
    apply()
    uvicorn.run(app,port=4444)

INFO:     Started server process [788506]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:57434 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:34618 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:46526 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:46540 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:47694 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56136 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56144 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56148 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56162 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56174 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56190 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56192 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56208 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [788506]
